In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

In [5]:
data = pd.read_csv('preprocessed.csv')
data = data.drop(['Unnamed: 0'] , axis = 1)
data

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,0,9839.64,170136.00,160296.36,0.00,0.00,0,0
1,0,1864.28,21249.00,19384.72,0.00,0.00,0,0
2,1,181.00,181.00,0.00,0.00,0.00,1,0
3,2,181.00,181.00,0.00,21182.00,0.00,1,0
4,0,11668.14,41554.00,29885.86,0.00,0.00,0,0
...,...,...,...,...,...,...,...,...
6362615,2,339682.13,339682.13,0.00,0.00,339682.13,1,0
6362616,1,6311409.28,6311409.28,0.00,0.00,0.00,1,0
6362617,2,6311409.28,6311409.28,0.00,68488.84,6379898.11,1,0
6362618,1,850002.52,850002.52,0.00,0.00,0.00,1,0


In [9]:
data.columns

Index(['type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest',
       'newbalanceDest', 'isFraud', 'isFlaggedFraud'],
      dtype='object')

In [11]:
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data.to_numpy())
data_scaled = pd.DataFrame(data_scaled , columns = data.columns)
data_scaled.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,0.00,0.000106,0.002855,0.003233,0.000000,0.0,0.0,0.0
1,0.00,0.000020,0.000357,0.000391,0.000000,0.0,0.0,0.0
2,0.25,0.000002,0.000003,0.000000,0.000000,0.0,1.0,0.0
3,0.50,0.000002,0.000003,0.000000,0.000059,0.0,1.0,0.0
4,0.00,0.000126,0.000697,0.000603,0.000000,0.0,0.0,0.0


In [12]:
data_scaled.describe()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,4.215772e-01,1.945599e-03,1.399484e-02,1.724540e-02,3.091721e-03,3.439269e-03,1.290820e-03,2.514687e-06
std,3.728388e-01,6.532045e-03,4.847261e-02,5.897038e-02,9.547833e-03,1.031539e-02,3.590480e-02,1.585775e-03
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,1.448374e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.000000e-01,8.099034e-04,2.384491e-04,0.000000e+00,3.727521e-04,6.026781e-04,0.000000e+00,0.000000e+00
75%,5.000000e-01,2.257778e-03,1.801042e-03,2.909313e-03,2.648861e-03,3.121768e-03,0.000000e+00,0.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [13]:
data_x = data_scaled.drop(['isFraud'] , axis = 1)
data_y = data_scaled['isFraud']
x_train , x_test , y_train , y_test = train_test_split(data_x , data_y , test_size=0.9)
print(x_train.shape , x_test.shape , y_train.shape  , y_test.shape)

(636262, 7) (5726358, 7) (636262,) (5726358,)


In [20]:
model = keras.Sequential([
    keras.layers.Dense(8, activation='relu', input_shape=(7,)),
    keras.layers.Dense(16 , activation = 'relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='mean_squared_error' , metrics = ['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 8)                 64        
                                                                 
 dense_13 (Dense)            (None, 16)                144       
                                                                 
 dense_14 (Dense)            (None, 1)                 17        
                                                                 
Total params: 225 (900.00 Byte)
Trainable params: 225 (900.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.fit(x_train , y_train , epochs = 5 , validation_split= 0.2)

Epoch 1/5
15907/15907 [==============================] - 19s 1ms/step - loss: 0.0027 - accuracy: 0.9987 - val_loss: 0.0013 - val_accuracy: 0.9987
Epoch 2/5
15907/15907 [==============================] - 21s 1ms/step - loss: 0.0011 - accuracy: 0.9988 - val_loss: 9.6698e-04 - val_accuracy: 0.9990
Epoch 3/5
15907/15907 [==============================] - 19s 1ms/step - loss: 8.3758e-04 - accuracy: 0.9991 - val_loss: 7.9634e-04 - val_accuracy: 0.9992
Epoch 4/5
15907/15907 [==============================] - 20s 1ms/step - loss: 7.5503e-04 - accuracy: 0.9992 - val_loss: 8.0162e-04 - val_accuracy: 0.9992
Epoch 5/5
15907/15907 [==============================] - 20s 1ms/step - loss: 7.3950e-04 - accuracy: 0.9992 - val_loss: 7.4987e-04 - val_accuracy: 0.9992


In [22]:
model.evaluate(x_test , y_test)

178949/178949 [==============================] - 148s 829us/step - loss: 7.3992e-04 - accuracy: 0.9992


[0.0007399229216389358, 0.999197244644165]

In [23]:
model.save('ANN')

INFO:tensorflow:Assets written to: ANN\assets


INFO:tensorflow:Assets written to: ANN\assets
